In [1]:
import tensorflow as tf
tf.config.threading.set_intra_op_parallelism_threads(6)
tf.config.threading.set_inter_op_parallelism_threads(6)


2024-01-28 19:20:14.785329: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 19:20:14.809577: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 19:20:14.810114: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 19:20:15.320474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import scipy.io
import numpy as np
import mne
import os
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [ ]:
# download_url = [
#     "https://physionet.org/static/published-projects/eegmmidb/eeg-motor-movementimagery-dataset-1.0.0.zip"
# ]

# for i in download_url:
#     os.system("wget --no-check-certificate -P /mnt/Data/Data/hgd "+i)

# target_directory = "/mnt/Data/Data/hgd/"

# # Full path to the zip files
# zip_file1 = os.path.join(target_directory, "eeg-motor-movementimagery-dataset-1.0.0.zip")

# # Unzip the files into the target directory
# os.system(f"unzip -o {zip_file1} -d {target_directory}")

In [2]:
import mne
file = '/mnt/Data/Data/hgd/files/S001/S001R01.edf'
# Step 1: Load the Data
# Replace 'your_file.edf' with the path to your EDF file
raw = mne.io.read_raw_edf(file, preload=True)

# Step 2: Preprocessing

# Filtering - Apply band-pass filter (1 Hz to 40 Hz is common for EEG)
raw.filter(0.1, 50., fir_design='firwin')

# Artifact Removal - Using ICA (Independent Component Analysis)
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)
raw = ica.apply(raw)
print(raw.info.ch_names)
# Step 3: Epoching Data Based on Events
raw.pick_channels(['C5..','C3..','C1..','Cz..','C2..','C4..','C6..'])
# Find events
events,event_ids = mne.events_from_annotations(raw)
print('events & eid',events,event_ids)
#event_id = {'event1': 1, 'event2': 2}  

# Define epochs parameters
tmin = -0.2  # Start time before event
tmax = 0.5   # End time after event
baseline = (None, 0)  # Means no baseline correction

# Extract epochs
epochs = mne.Epochs(raw, events[0], event_ids, tmin, tmax, baseline=baseline, preload=True)

Extracting EDF parameters from /mnt/Data/Data/hgd/files/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 5281 samples (33.006 s)

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 20 components


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 0.6s.
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components
['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['T0']
events & eid [[0 0 1]] {'T0': 1}


ValueError: events must be of shape (N, 3), got (3,)